Example URL: http://localhost:8888/notebooks/scatterplot.ipynb?patientset=28726&user=demo-user&password=demo-user&concept_x=1341006&concept_y=SNOMEDCT/Sanofi6

In [1]:
import transmart as tm
import pandas as pd
import numpy as np
import time
from pandas.io.json import json_normalize
from IPython.display import HTML

pd.set_option('max_colwidth', 1000)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

print('transmart python client version: {}'.format(tm.__version__))

transmart python client version: 0.1.3


With Run all the following can give issues with Javascript for D3 or grabbing paramters from URL not being executed yet

In [2]:
import Args

<IPython.core.display.Javascript object>

In [3]:
import d3_lib

In [4]:
HTML('<script src="lib/d3/d3.min.js"></script>')

In [5]:
HTML(d3_lib.set_styles(['basic_axis','basic_line','basic_scatter','force_directed_graph','day-hr-heatmap']))

In real setting a token would need to be passed, rather than user/password.

In [6]:
args = Args.Parse(patientset=int, user=str, password=str, concept_x=str, concept_y=str)
args

Args(concept_x='1341006', concept_y='SNOMEDCT/Sanofi6', password='demo-user', patientset=28726, user='demo-user', extra_=[])

In [7]:
import getpass
from transmart import TransmartApi

api = tm.TransmartApi(
    host = 'http://transmart.thehyve.net',
    user = args.user,  # input('Username:'),
    password = args.password,  # getpass.getpass('Password:'),
    api_version = 2,
    print_urls = False)

api.access()

'Connected successfully'

In [8]:
patient_set_id = args.patientset
# patients = api.get_patients(patientSet = patient_set_id)
# patients

In [9]:
observations = api.get_observations(patientSet = patient_set_id)
# observations

In [10]:
# pd.DataFrame(observations['concept.conceptPath'].unique())

In [11]:
concept_x = args.concept_x
concept_y = args.concept_y

In [12]:
observations_filtered = observations[(observations['concept.conceptCode'] == concept_x) | \
                                     (observations['concept.conceptCode'] == concept_y)]
# observations_filtered

In [13]:
observations_pivot_table = pd.pivot_table(observations_filtered,
                                          values='numericValue',
                                          index=['patient.id'],
                                          columns=['concept.conceptCode'],
                                          aggfunc=np.average
                                         ).dropna()
# observations_pivot_table

In [14]:
observations_pivot_table = observations_pivot_table.rename(index=str, columns={concept_x: "x", concept_y: "y"})
observations_array_of_dicts = observations_pivot_table.to_dict(orient='records')
# observations_array_of_dicts

In [16]:
HTML(d3_lib.draw_graph('basic_scatter',{'data': observations_array_of_dicts}))